In [1]:
#bagofwords
import pandas as pd
train=pd.read_csv("labeledTrainData.tsv",delimiter="\t",quoting=3,header=0)


In [2]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
train.review[0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [4]:
from bs4 import BeautifulSoup

In [5]:
import re 
import nltk
from nltk.corpus import stopwords

In [6]:
#cleaning a review

def convert_review(review):
    #using beautiful soup remove tags
    review_tagsremoved=BeautifulSoup(review,"lxml").get_text()
    #removing punctuations
    N_review=re.sub("[^a-zA-Z]"," ",review_tagsremoved)
    n_review=N_review.lower().split()
    stops=set(stopwords.words("english"))
    useful_words=[w for w in n_review if not w in stops]
    return(" ".join(useful_words))

    

In [7]:
train['clean_review']=train['review'].apply(lambda x: convert_review(x))

In [8]:
train.columns

Index(['id', 'sentiment', 'review', 'clean_review'], dtype='object')

In [9]:
#using count of each word as a feature 

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
vectorizer= CountVectorizer(analyzer="word",tokenizer=None,preprocessor=None,stop_words=None,max_features=5000)

In [12]:
features=vectorizer.fit_transform(train['clean_review'].tolist())

In [13]:
import numpy as np
features=features.toarray()

In [17]:
features.size

125000000

In [18]:
vocab=vectorizer.get_feature_names()

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
forest=RandomForestClassifier(n_estimators=500)
forest=forest.fit(features,train['sentiment'])

In [21]:
test=pd.read_csv("testData.tsv",delimiter="\t",quoting=3,header=0)

In [22]:
test["clean_review"]=test['review'].apply(lambda x: convert_review(x))
        

In [23]:
test_reviews=test['clean_review'].tolist()
test_features=vectorizer.transform(test_reviews)
test_features=test_features.toarray()
result=forest.predict(test_features)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("BOG_1.csv",index=False,quoting=3)
#0.8565

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer(analyzer="word",tokenizer=None,preprocessor=None,stop_words=None,max_features=5000)
features=vectorizer.fit_transform(train['clean_review'].tolist())
features=features.toarray()
forest=RandomForestClassifier(n_estimators=500)
forest=forest.fit(features,train['sentiment'])
test_reviews=test['clean_review'].tolist()
test_features=vectorizer.transform(test_reviews)
test_features=test_features.toarray()
result=forest.predict(test_features)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("BOG_2.csv",index=False,quoting=3)
#0.853

In [28]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer= HashingVectorizer(analyzer="word",tokenizer=None,preprocessor=None,stop_words=None,n_features=5000)
features=vectorizer.fit_transform(train['clean_review'].tolist())
features=features.toarray()
forest=RandomForestClassifier(n_estimators=500)
forest=forest.fit(features,train['sentiment'])
test_reviews=test['clean_review'].tolist()
test_features=vectorizer.transform(test_reviews)
test_features=test_features.toarray()
result=forest.predict(test_features)
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("BOG_3.csv",index=False,quoting=3)
#0.84